In [4]:
import time
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random

ol = Overlay('/home/xilinx/jupyter_notebooks/sadd/sadd.bit') # check this path
t_before_bitstream = time.time()
ol.download() # this downloads your bitstream into FPGA
t_after_bitstream = time.time()
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')
dma1 = ol.streamAdd.sadd_dma1 # first dma
dma2 = ol.streamAdd.sadd_dma2 # second dma
sadd_ip = MMIO(0x43c00000, 0x10000) # we got this address from 
xlnk = Xlnk()

1.412102460861206 seconds to program bitstream 


In [5]:
length = 5

in_buffer1 = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer 1
in_buffer2 = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer 1
out_buffer = xlnk.cma_array(shape=(length,), dtype=np.int32) # output buffer

samples = random.sample(range(0, length), length)
np.copyto(in_buffer1, samples)
samples = random.sample(range(0, length), length)
np.copyto(in_buffer2, samples)

sadd_ip.write(0x10, length) # we got this address from vivado
t_start = time.time()
dma1.sendchannel.transfer(in_buffer1)
dma2.sendchannel.transfer(in_buffer2)
dma1.recvchannel.transfer(out_buffer)
dma1.sendchannel.wait()
dma2.sendchannel.wait()
dma1.recvchannel.wait()
t_stop = time.time()
in_buffer1.close()
in_buffer2.close()
out_buffer.close()
print('Hardware execution time: ', t_stop-t_start)
for i in range(0, length):
    print('{}+{} = {}'.format(in_buffer1[i], in_buffer2[i], out_buffer[i]))

Hardware execution time:  0.002737283706665039
3+1 = 4
4+0 = 4
2+4 = 6
1+3 = 4
0+2 = 2


In [ ]:
ol.ip_dict

In [ ]:
ol.hierarchy_dict.

In [ ]:
import numpy as np

X = np.linspace(-np.pi, np.pi, 256, endpoint=True)
C, S = np.cos(X), np.sin(X)